In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [3]:
heart_data = pd.read_csv('/content/heart_disease_data.csv')

In [4]:
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
heart_data.isnull().sum()

,0
age,0
sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0
oldpeak,0


In [6]:
X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']

In [7]:
print(X)

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  
0        0   0     1  
1        0   0     2  
2        2   0    

Model_Selection

In [8]:
#comparing the model with default hyperparameter using Cross Validation

In [9]:
models = [LogisticRegression(max_iter=1000), SVC(kernel='linear'), KNeighborsClassifier(), RandomForestClassifier(random_state=0)]

In [10]:
def compare_models_cross_validation():
  for model in models:
    cv_score = cross_val_score(model, X, Y, cv=5)
    mean_accuracy = sum(cv_score)/len(cv_score)
    mean_accuracy = mean_accuracy*100
    mean_accuracy = round(mean_accuracy, 2)

    print('Cross Validation accuracies for the', model, '= ', cv_score)
    print('Accuracy % of the ', model, "=" ,mean_accuracy)
    print("****************************************")

In [11]:
compare_models_cross_validation()

Cross Validation accuracies for the LogisticRegression(max_iter=1000) =  [0.80327869 0.86885246 0.85245902 0.86666667 0.75      ]
Accuracy % of the  LogisticRegression(max_iter=1000) = 82.83
****************************************
Cross Validation accuracies for the SVC(kernel='linear') =  [0.81967213 0.8852459  0.80327869 0.86666667 0.76666667]
Accuracy % of the  SVC(kernel='linear') = 82.83
****************************************
Cross Validation accuracies for the KNeighborsClassifier() =  [0.60655738 0.6557377  0.57377049 0.73333333 0.65      ]
Accuracy % of the  KNeighborsClassifier() = 64.39
****************************************
Cross Validation accuracies for the RandomForestClassifier(random_state=0) =  [0.85245902 0.90163934 0.81967213 0.81666667 0.8       ]
Accuracy % of the  RandomForestClassifier(random_state=0) = 83.81
****************************************


Comparing the models with diffrent Hyperparameter values using GridSearchCV

In [12]:
models_list = [LogisticRegression(max_iter=10000), SVC(), KNeighborsClassifier(), RandomForestClassifier(random_state=0)]

In [13]:
#creating the dictionary that contains hyperparameter values for above mentioned models

In [14]:
model_hyperparameters = {
    'log_reg_hyperparameters': {
        'C' : [1,5,10,20]
    },
    'svc_hyperparameters' : {
        'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
        'C' : [1,5,10,20]
    },
    'KNN_hyperparameters' : {
        'n_neighbors' : [3,5,10]
    },
    'random_forest_hyperparameters' : {
        'n_estimators' : [10, 20,50,100]
    },
}

In [15]:
type(model_hyperparameters)

dict

In [16]:
#dict becuase we are passing more than one parameters so we are using key values pair

In [17]:
model_keys = list(model_hyperparameters.keys())

In [18]:
print(model_keys)

['log_reg_hyperparameters', 'svc_hyperparameters', 'KNN_hyperparameters', 'random_forest_hyperparameters']


In [19]:
model_keys[0]

'log_reg_hyperparameters'

In [20]:
#0 because we will get them through indexes

Applying Grid Search

In [21]:
def ModelSelection(list_of_models, hyperparameters_dictionary):
  #for outcomes
  result = []

  #0 because of first index
  i = 0

  for model in list_of_models:

    key = model_keys[i]

    params = hyperparameters_dictionary[key]

    i += 1

    print(model)
    print(params)
    print("**************************")

    classifier = GridSearchCV(model, params, cv=5)

    #fit the model
    classifier.fit(X,Y)

    result.append({
        'model_used': model,
        'highest_score': classifier.best_score_,
        'best_hyperparameters': classifier.best_params_
    })

  result_dataframe = pd.DataFrame(result, columns=['model_used', 'highest_score', 'best_hyperparameters'])

  return result_dataframe

In [22]:
ModelSelection(models_list, model_hyperparameters)

LogisticRegression(max_iter=10000)
{'C': [1, 5, 10, 20]}
**************************
SVC()
{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}
**************************
KNeighborsClassifier()
{'n_neighbors': [3, 5, 10]}
**************************
RandomForestClassifier(random_state=0)
{'n_estimators': [10, 20, 50, 100]}
**************************


,model_used,highest_score,best_hyperparameters
0,LogisticRegression(max_iter=10000),0.831585,{'C': 5}
1,SVC(),0.828306,"{'C': 1, 'kernel': 'linear'}"
2,KNeighborsClassifier(),0.643880,{'n_neighbors': 5}
3,RandomForestClassifier(random_state=0),0.838087,{'n_estimators': 100}


Random Forest Classifier with n_estimators = 100 has the highest accuracy